In [12]:
#concatenate all sql files
from zipfile import ZipFile
import os

# Define the path for the uploaded zip file and the directory to extract its contents
zip_path = "C:\\Users\\anany\\OneDrive\\Desktop\\Anti-pattern\\Anti-pattern revised.zip"
extract_dir = "C:\\Users\\anany\\OneDrive\\Desktop\\Anti-pattern\\Anti-pattern"

# Create a directory to extract the zip file contents
os.makedirs(extract_dir, exist_ok=True)

# Extract the zip file
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List the extracted files to confirm extraction
extracted_files = os.listdir(extract_dir)
extracted_files
# Path for the final concatenated SQL file
concatenated_sql_path = 'C:\\Users\\anany\\OneDrive\\Desktop\\Anti-pattern\\concatenated_sql_files.sql'

# Concatenate all SQL files into one
with open(concatenated_sql_path, 'w') as outfile:
    for fname in extracted_files:
        with open(os.path.join(extract_dir, fname)) as infile:
            for line in infile:
                outfile.write(line)
            # Ensure there's a newline between files in case the last line of a file doesn't end with one
            outfile.write('\n')

concatenated_sql_path


UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 106: character maps to <undefined>

In [13]:
#converting sql to csv
import csv

# Path for the CSV file to be created
csv_path = 'C:\\Users\\anany\\OneDrive\\Desktop\\Anti-pattern\\sql_statements.csv'

# Read the SQL file, split statements by semicolon, and write each statement as a row in the CSV
with open(concatenated_sql_path, 'r') as sql_file:
    sql_content = sql_file.read()

# Splitting by semicolon to get individual statements; this assumes that each statement ends with a semicolon
sql_statements = sql_content.split(';')

# Remove any leading/trailing whitespace and filter out empty statements
sql_statements = [stmt.strip() for stmt in sql_statements if stmt.strip()]

# Write to CSV
with open(csv_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    for statement in sql_statements:
        writer.writerow([statement])  # Writing each statement as a row in the CSV

csv_path


'C:\\Users\\anany\\OneDrive\\Desktop\\Anti-pattern\\sql_statements.csv'

In [4]:
#removing duplicate value
import pandas as pd

# Load the CSV file into a DataFrame
csv_path_uploaded = '/mnt/data/Anti_pattern.csv'
df = pd.read_csv(csv_path_uploaded, header=None)  # Assuming the CSV has a single column and no header

# Remove duplicate rows
df_unique = df.drop_duplicates()

# Path for the deduplicated CSV file
deduplicated_csv_path = '/mnt/data/anti_pattern_deduplicated.csv'

# Save the deduplicated DataFrame to a new CSV file
df_unique.to_csv(deduplicated_csv_path, index=False, header=False)

deduplicated_csv_path


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/Anti_pattern.csv'

In [5]:
#naturer error findings
# Enhance the function to provide more detailed error analysis
def detailed_sql_syntax_check(statement):
    # Initialize error details
    error_exists = 'No'
    error_type = ''
    error_location = ''

    # Check for common SQL start keywords
    common_starts = ['SELECT', 'INSERT', 'UPDATE', 'DELETE', 'CREATE', 'ALTER', 'DROP']
    if not any(statement.strip().upper().startswith(keyword) for keyword in common_starts):
        error_exists = 'Yes'
        error_type = 'Uncommon start'
        error_location = 'Start of statement'
    # check for common sql keywords in the middle
    common_middle=['from', 'where','as','and','or']
    
    
    
    
    # Check for mismatched parentheses
    open_parentheses = statement.count('(')
    close_parentheses = statement.count(')')
    if open_parentheses != close_parentheses:
        error_exists = 'Yes'
        error_type += 'Mismatched parentheses' + ('; ' if error_type else '')
        error_location += 'Parentheses count mismatch' + ('; ' if error_location else '')

    return pd.Series([error_exists, error_type, error_location])

# Apply the detailed syntax check function to each SQL statement
df_deduplicated[['Error_Exists', 'Error_Type', 'Error_Location']] = df_deduplicated['SQL_Statement'].apply(detailed_sql_syntax_check)

# Filter to show only rows with errors for demonstration, though all rows will be saved
errors_detailed_df = df_deduplicated[df_deduplicated['Error_Exists'] == 'Yes']

# Define path for the CSV with detailed error analysis
detailed_errors_csv_path = '/mnt/data/sql_errors_detailed_analysis.csv'

# Save the DataFrame with detailed error analysis to a new CSV file
df_deduplicated.to_csv(detailed_errors_csv_path, index=False)

detailed_errors_csv_path, errors_detailed_df.head()


NameError: name 'df_deduplicated' is not defined

In [6]:
#corrected statements
# Function to attempt a basic correction of SQL statements
def correct_sql_statement(statement):
    # Count the number of opening and closing parentheses
    open_parentheses = statement.count('(')
    close_parentheses = statement.count(')')

    # Attempt to correct mismatched parentheses
    while open_parentheses > close_parentheses:
        statement += ')'  # Add a closing parenthesis at the end
        close_parentheses += 1
    while close_parentheses > open_parentheses:
        if statement.endswith(')'):
            statement = statement[:-1]  # Remove a closing parenthesis from the end
            close_parentheses -= 1
        else:
            break  # Stop if the statement does not end with a closing parenthesis

    return statement

# Apply the correction function to each SQL statement and create a new column for corrected statements
df_deduplicated['Corrected_SQL_Statement'] = df_deduplicated['SQL_Statement'].apply(correct_sql_statement)

# Define path for the CSV with corrected SQL statements
corrected_sql_csv_path = '/mnt/data/sql_corrected_statements.csv'

# Save the DataFrame with corrected SQL statements to a new CSV file
df_deduplicated.to_csv(corrected_sql_csv_path, index=False)

corrected_sql_csv_path


NameError: name 'df_deduplicated' is not defined

In [7]:
#injecting error
# Ensure all SQL statements are treated as strings to prevent the previous error
full_corrected_sql_df['Corrected_SQL_Statement'] = full_corrected_sql_df['Corrected_SQL_Statement'].astype(str)

# Re-apply the function to potentially introduce errors into the SQL queries
full_corrected_sql_df[['Modified_SQL', 'Error_Type', 'Error_Location']] = full_corrected_sql_df['Corrected_SQL_Statement'].apply(
    lambda x: pd.Series(maybe_introduce_error(x)))

# Re-add the column to indicate whether the query is wrong
full_corrected_sql_df['Is_Wrong'] = full_corrected_sql_df['Error_Type'].apply(lambda x: 'Yes' if x != 'No modification' else 'No')

# Re-define path for the full CSV with randomly introduced errors
full_modified_sql_csv_path = '/mnt/data/full_sql_with_random_errors.csv'

# Re-save the DataFrame with randomly introduced errors to a new CSV file
full_corrected_sql_df.to_csv(full_modified_sql_csv_path, index=False)

full_modified_sql_csv_path


NameError: name 'full_corrected_sql_df' is not defined

In [8]:
# Define a function to validate SQL statements and attempt to correct them if possible
def validate_and_correct_sql(statement):
    try:
        # Pretend to parse the SQL statement to detect errors (simulation, as actual parsing is not feasible here)
        # This is a placeholder for actual SQL parsing logic, which would require a proper SQL parser
        # For demonstration, we assume the statement is correct and return it as is
        return statement, 'No', 'N/A', 'N/A'  # No error found
    except Exception as e:
        # If an error is detected, attempt to correct it
        # This is a simplified correction mechanism and may not cover all SQL syntax errors
        corrected_statement = statement.replace(",,", ",")  # Example correction: removing double commas
        return corrected_statement, 'Yes', str(e), 'Corrected double commas'  # Error found and corrected

# Apply the validation and correction function to each SQL statement
hj_updated_df[['Corrected_Statement', 'Error_Found', 'Error_Description', 'Correction_Applied']] = hj_updated_df['statement'].apply(
    lambda x: pd.Series(validate_and_correct_sql(x)))

# Display the first few rows to verify the corrections
hj_updated_df.head()


NameError: name 'hj_updated_df' is not defined

In [9]:
#finding the error
import sqlparse
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML

def is_select_statement(parsed):
    # This function checks if the parsed statement is a SELECT statement
    for token in parsed.tokens:
        if token.ttype is DML and token.value.upper() == 'SELECT':
            return True
    return False

def find_errors(sql):
    # Parse the SQL statement
    parsed = sqlparse.parse(sql)[0]
    
    # Check if it's a SELECT statement
    if not is_select_statement(parsed):
        return "Not a SELECT statement", "N/A"
    
    # Look for common errors like missing FROM clause
    from_clause_found = False
    for token in parsed.tokens:
        if from_clause_found:
            break
        if isinstance(token, IdentifierList):
            for identifier in token.get_identifiers():
                if str(identifier).upper().startswith('FROM '):
                    from_clause_found = True
                    break
        elif isinstance(token, Identifier) and str(token).upper().startswith('FROM '):
            from_clause_found = True
    
    if not from_clause_found:
        return "Missing FROM clause", "N/A"
    
    # No errors found
    return "No errors", "N/A"

# Example usage
sql_statements = [
    "SELECT * FROM table_name",  # Correct SQL
    "SELECT *",  # Missing FROM clause
    "UPDATE table_name SET column=value"  # Not a SELECT statement
]

for sql in sql_statements:
    error, correction = find_errors(sql)
    print(f"SQL: {sql}\nError: {error}\nCorrection: {correction}\n")

ModuleNotFoundError: No module named 'sqlparse'